# Guide to neo4j with python

At the hearth of Neo4j lies Cypher, the query language you use to interact with a Neo4j database. While this guide does not require you to be a seasoned Cypher querier, it is going to be easier to focus on the Python-specific bits if you already know some Cypher. For this reason, although this guide does also provide a gentle introduction to Cypher along the way, consider checking out [Getting started → Cypher](https://neo4j.com/docs/getting-started/cypher-intro/?utm_medium=PaidSearch&utm_source=google&utm_campaign=GDB&utm_content=EMEA-X-Awareness-GDB-Text&gclid=CjwKCAiAvdCrBhBREiwAX6-6UoIDTEMkwSDABQLQeIr12PK5XynwqS6mGynXPS62O190iHmgFyBKshoCqEUQAvD_BwE) for a more detailed walkthrough of graph databases modelling and querying if this is your first approach. You may then apply that knowledge while following this guide to develop your Python application.


## Installation

In [ ]:
!pip install neo4j

## Connecting to the datbase

In [ ]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "<URI for Neo4j database>"
AUTH = ("<Username>", "<Password>")

driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()

## Write to the database

Execute a Cypher statement with the method Driver.execute_query(). Do not hardcode or concatenate parameters: use placeholders and specify the parameters as keyword arguments.

### Insert

#### Insert nodes

In [ ]:
summary = driver.execute_query(
    """CREATE (:Person {name: $name, age: $age}),
    (:Person {name:$name_2, age: $age_2})
    """,
    name="Alice",
    age=42,
    name_2="Bob",
    age_2=41,
    database_="neo4j",
).summary
print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

#### Insert Relationships

In [ ]:
summary = driver.execute_query(
    """MATCH (alice:Person {name: $name}), (bob:Person {name:$name_2})
    CREATE (bob)-[:SENT_MESSAGE {date: $date}]->(alice)
    """,
    name="Alice",
    name_2="Bob",
    date="2023-12-09",
    database_="neo4j",
).summary

print("Created {relationships_created} relationshpis in {time} ms.".format(
    relationships_created=summary.counters.relationships_created,
    time=summary.result_available_after
))

In [ ]:
summary = driver.execute_query(
    """CREATE (person1:Person {name: $name, age: $age}),
    (person2:Person {name:$name_2, age: $age_2}),
    (person1)-[:SENT_MESSAGE {date: $date}]->(person2),
    (person1)<-[:SENT_MESSAGE {date: $date_2}]-(person2)
    """,
    name="Jim",
    age=50,
    name_2="Jane",
    age_2=44,
    date="2023-12-08",
    date_2="2023-12-09",
    database_="neo4j",
).summary

print("Created {nodes_created} nodes and {relationships_created} relatoinships in {time} ms.".format(
    relationships_created=summary.counters.relationships_created,
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

### Update

In [ ]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person {name: $name})
    SET p.age = $age
    """, name="Alice", age=42,
    database_="neo4j",
)
print(f"Query counters: {summary.counters}.")

In [ ]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person {name: $name})
    MERGE (p)-[:KNOWS]->(:Person {name: $friend})
    """, name="Alice", friend="Bob",
    database_="neo4j",
)
print(f"Query counters: {summary.counters}.")

## Query the database

Execute a Cypher statement with the method Driver.execute_query(). Do not hardcode or concatenate parameters: use placeholders and specify the parameters as keyword arguments.

In [ ]:
# Get the name of all 42 year-olds
records, summary, keys = driver.execute_query(
    "MATCH (p:Person {age: $age}) RETURN p.name AS name",
    age=42,
    database_="neo4j",
)

# Loop through results and do something with them
for person in records:
    print(person)

# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after,
))

## Delete from the database

To remove a node and any relationship attached to it, use the Cypher clause DETACH DELETE:

In [ ]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person {name: $name})
    DETACH DELETE p
    """, name="Alice",
    database_="neo4j",
)
print(f"Query counters: {summary.counters}.")

## Query Parameters

Do not hardcode or concatenate parameters directly into queries. Instead, always use placeholders and specify the Cypher parameters, as shown in the previous examples. This is for:

1. performance benefits: Neo4j compiles and caches queries, but can only do so if the query structure is unchanged;

2. security reasons: see protecting against Cypher injection.

Query parameters can be passed either as several keyword arguments, or grouped together in a dictionary as value to the parameters_ keyword argument. In case of mix, keyword-argument parameters take precedence over dictionary ones.

Pass query parameters as keyword arguments:

In [ ]:
driver.execute_query(
    "MERGE (:Person {name: $name})",
    name="Alice", age=42,
    database_="neo4j",
)

Pass query parameters in a dictionary:

In [ ]:
parameters = {
    "name": "Alice",
    "age": 42
}
driver.execute_query(
    "MERGE (:Person {name: $name})",
    parameters_=parameters,
    database_="neo4j",
)

## Transform Query Result

You can transform a query’s result into a different data structure using the result_transformer_ argument. The driver provides built-in methods to transform the result into a pandas dataframe or into a graph, but you can also craft your own transformer.

For more information, see [Manipulate query results](https://neo4j.com/docs/python-manual/current/transformers/).

### Result as a list

By default, Driver.execute_query() returns an EagerResult object.

The EagerResult object has the following:
1. The result records as a list, so it is easy to loop through them.
2. A summary of execution, with metadata and information about the result.
3. The keys available in the returned rows.

In [ ]:
records, summary, keys = driver.execute_query(
    "MATCH (a:Person) RETURN a.name AS name, a.age AS age",
    database_="neo4j",
)
for person in records:
    print(person)
    # person["name"] or person["age"] are also valid

# Some summary information
print("Query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after
))

print(f"Available keys are {keys}")  # ['name', 'age']

### Transform to pandas DataFrame

In [ ]:
import neo4j

pandas_df = driver.execute_query(
    "UNWIND range(1, 10) AS n RETURN n, n+1 AS m",
    database_="neo4j",
    result_transformer_=neo4j.Result.to_df
)
print(type(pandas_df))

pandas_df

### Transform to graph

The driver can transform the result into a collection of graph objects. To achieve this, use the .execute_query() keyword argument result_transformer_ and set it to neo4j.Result.graph. To make the most out of this method, your query should return a graph-like result instead of a single column. The graph transformer exposes the properties nodes and relationships, which are set views into Node and Relationship objects.

You can use the graph format for further processing or to visualize the query result. An example implementation that uses the pyvis library to draw the graph is below.

In [ ]:
!pip install pyvis==0.3.1

In [ ]:
import pyvis
from neo4j import GraphDatabase
import neo4j
from IPython.core.display import display, HTML

URI = "<URI for Neo4j database>"
AUTH = ("<Username>", "<Password>")

def main():
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        # Create some friends
        input_list = [("Arthur", "Guinevre"),
                      ("Arthur", "Lancelot"),
                      ("Arthur", "Merlin")]
        driver.execute_query("""
            UNWIND $pairs AS pair
            MERGE (a:Person {name: pair[0]})
            MERGE (a)-[:KNOWS]->(friend:Person {name: pair[1]})
            """, pairs=input_list,
            database_="neo4j",
        )

        # Create a film
        driver.execute_query("""
            MERGE (film:Film {title: $title})
            MERGE (liker:Person {name: $person_name})
            MERGE (liker)-[:LIKES]->(film)
            """, title="Wall-E", person_name="Arthur",
            database_="neo4j",
        )

        # Query to get a graphy result
        graph_result = driver.execute_query("""
            MATCH (a:Person {name: $name})-[r]-(b)
            RETURN a, r, b
            """, name="Arthur",
            result_transformer_=neo4j.Result.graph,
        )

        # Draw graph
        nodes_text_properties = {  # what property to use as text for each node
            "Person": "name",
            "Film": "title",
        }
        visualize_result(graph_result, nodes_text_properties)


def visualize_result(query_graph, nodes_text_properties):
    visual_graph = pyvis.network.Network(notebook=True, cdn_resources='in_line')

    for node in query_graph.nodes:
        node_label = list(node.labels)[0]
        node_text = node[nodes_text_properties[node_label]]
        visual_graph.add_node(node.element_id, node_text, group=node_label)

    for relationship in query_graph.relationships:
        visual_graph.add_edge(
            relationship.start_node.element_id,
            relationship.end_node.element_id,
            title=relationship.type
        )

    visual_graph.show('network.html')
    display(HTML('network.html'))

main()

## Full example

In [ ]:
from neo4j import GraphDatabase


URI = "<URI for Neo4j database>"
AUTH = ("<Username>", "<Password>")

people = [{"name": "Alice", "age": 42, "friends": ["Bob", "Peter", "Anna"]},
          {"name": "Bob", "age": 19},
          {"name": "Peter", "age": 50},
          {"name": "Anna", "age": 30}]

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    try:
        records, summary, keys = driver.execute_query("""
            MATCH (p:Person)
            DETACH DELETE p
            """,
            database_="neo4j",
        )
        print(f"Query counters: {summary.counters}.")

        # Create some nodes
        for person in people:
            records, summary, keys = driver.execute_query(
                "MERGE (p:Person {name: $person.name, age: $person.age})",
                person=person,
                database_="neo4j",
            )

        # Create some relationships
        for person in people:
            if person.get("friends"):
                records, summary, keys = driver.execute_query("""
                    MATCH (p:Person {name: $person.name})
                    UNWIND $person.friends AS friend_name
                    MATCH (friend:Person {name: friend_name})
                    MERGE (p)-[:KNOWS]->(friend)
                    """, person=person,
                    database_="neo4j",
                )

        # Retrieve Alice's friends who are under 40
        records, summary, keys = driver.execute_query("""
            MATCH (p:Person {name: $name})-[:KNOWS]-(friend:Person)
            WHERE friend.age < $age
            RETURN friend
            """, name="Alice", age=40,
            routing_="r",
            database_="neo4j",
        )
        # Loop through results and do something with them
        for record in records:
            print(record)
        # Summary information
        print("The query `{query}` returned {records_count} records in {time} ms.".format(
            query=summary.query, records_count=len(records),
            time=summary.result_available_after
        ))

    except Exception as e:
        print(e)
        # further logging/processing